# Bike Angels Points Collecting

## 2945 Final Project - Paul Kottering 

Multiple approaches have been attempted at trying to use the large amounts of data generated through a bike-sharing system to try and come up with a method for ensuring the system is sustainable by re-balancing the bike distribution. The problem arises when asymmetric demand can leave certain stations empty and certain stations full thereby preventing users from picking up a bike and dropping off bikes at these locations. Research is often focused on countering this asymmetric demand with trucks to move sets of bikes, and more recently points have been used to affect the behavior of riders to pickup/dropoff at stations to improve the distribution. In this project we take this idea further and see whether it is possible/plausible to redistribute the whole system based on individual users earning money by moving bikes around individually. In order to answer this question we must first figure out what is the most amount of points a rider can achieve in a specific time interval. This is what the method below calculates using a Integer Linear Programming approach. 

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
from itertools import product,combinations
from gurobipy import *
import urllib.request, json 
import pandas as pd

Collect Distance Data From Folder

In [8]:
CycleTime = np.loadtxt("CleanData/bicyclingmatrix.txt", dtype=int)
DataStations = np.loadtxt("CleanData/bicycling2matrix.txt", dtype=int)[0,:][1:]
WalkingTime = np.loadtxt("CleanData/walkingmatrix.txt", dtype=int)

Collect Live Points Data 


In [9]:
with urllib.request.urlopen("https://layer.bicyclesharing.net/map/v1/nyc/stations") as url:
    data = json.loads(url.read().decode())
df = pd.DataFrame(data)
new = pd.DataFrame.from_dict(df['features'])
clean = pd.DataFrame(columns = ['long','lat','id','action','points'])
for i in range(1588):
    if 'bike_angels_action' in pd.DataFrame(new['features'].iloc[i])['properties'].index:
        if int(pd.DataFrame(new['features'].iloc[i])['properties']['station_id']) in DataStations:
            long = pd.DataFrame(new['features'].iloc[i])['geometry']['coordinates'][0]
            lat = pd.DataFrame(new['features'].iloc[i])['geometry']['coordinates'][1]
            station_id = int(pd.DataFrame(new['features'].iloc[i])['properties']['station_id'])
            action = pd.DataFrame(new['features'].iloc[i])['properties']['bike_angels_action']
            points = pd.DataFrame(new['features'].iloc[i])['properties']['bike_angels_points']
            clean = clean.append({'long': long,'lat':lat,'id':station_id,'action':action,'points':points},ignore_index=True)
reduce = clean[clean['lat'] < 40.766411]
reduce = reduce[reduce['lat'] > 40.701249]
reduce = reduce[reduce['long'] < -73.967222]

In [10]:
Stations = reduce.loc[:,'id'].values
Action = reduce.loc[:,'action'].values
Points = reduce.loc[:,'points'].values

In [11]:
ismem = np.argwhere(~np.isin(DataStations, Stations)).ravel()

In [12]:
CycleTime = np.delete(CycleTime, ismem, 0)
CycleTime = np.delete(CycleTime, ismem, 1)
DataStations = np.delete(DataStations, ismem, 0)
WalkingTime = np.delete(WalkingTime, ismem, 0)
WalkingTime = np.delete(WalkingTime, ismem, 1)
    
PickUpP = np.zeros(len(DataStations))
DropOffP = np.zeros(len(DataStations))

for i in range(len(DataStations)):
        j = np.where(Stations == DataStations[i]) 
        if Action[j] == 'take':
            PickUpP[i] = Points[j]
        if Action[j] == 'give':
            DropOffP[i] = Points[j]
        

In [22]:
CycleTime = CycleTime*0.5

Set Up Parameters of Model - Including the total time constraint and the start station

In [13]:
Start = int(np.where(DataStations == 236)[0])
MAX = 45*60
numstations = len(Stations)
cartesian_prod = list(product(range(numstations), range(numstations)))

Add Objective and Constraints to a Model - We also include a number of variables which can be passed into the routine in order to control the cutting methods and presolve parameters

In [20]:
def modeloptimize(C,L,Z,G,P):
    
    # Create a new model
    m = gp.Model("PointsTest")

    # Create variables - Add upper and lower bound    
    Cycle_x = m.addVars(cartesian_prod,lb=0, ub=1, vtype=GRB.INTEGER, name='RouteCycl')
    Walk_y = m.addVars(cartesian_prod,lb=0, ub=1, vtype=GRB.INTEGER, name='RouteWalk')

    # Set objective
    m.setObjective(gp.quicksum((PickUpP[i] + DropOffP[j])*Cycle_x[(i,j)] for i,j in cartesian_prod), GRB.MAXIMIZE)

    # Add constraint - Leave As Many Stations as Visit
    m.addConstrs((gp.quicksum(Cycle_x[(i,j)] - Walk_y[(j,i)] for i in range(numstations)) == 0 
                  for j in range(numstations)), name='Leave')

    # Add constraint - Cant pick up and drop off at same station 
    m.addConstrs((Cycle_x[(j,j)] == 0 for j in range(numstations)), name='CycleSame')

    # Add constraint - Cant pick up and drop off at same station 
    m.addConstrs((Walk_y[(j,j)] == 0 for j in range(numstations)), name='WalkSame')

    # Add constraint - Cant walk and cycle same route 
    m.addConstrs((Cycle_x[(i,j)] + Walk_y[(j,i)] <= 1 for i,j in cartesian_prod), name='NoReturn')

    # Add constraint - Visit As Many Stations as Leave
    m.addConstrs((gp.quicksum(Cycle_x[(i,j)] - Walk_y[(j,i)] for j in range(numstations)) == 0 
                  for i in range(numstations)), name='Visit')

    # Add constraint - Start Node
    m.addConstr(gp.quicksum(Cycle_x[(Start,j)] + Walk_y[(Start,j)] for j in range(numstations)) >= 1 , name='Once')

    # Add constraint - Dont drop off at place with pickup points
    m.addConstrs((gp.quicksum(Cycle_x[(i,j)]*PickUpP[j] for j in range(numstations)) == 0 
                  for i in range(numstations)), name='FalseDrop')

    # Add constraint - Dont pick up at place with dropoff points
    m.addConstrs((gp.quicksum(Cycle_x[(i,j)]*DropOffP[i] for i in range(numstations)) == 0 
                  for j in range(numstations)), name='FalsePick')

    # Add constraint - Max Time
    m.addConstr(gp.quicksum(Cycle_x[(i,j)]*CycleTime[(i,j)] + Walk_y[(i,j)]*WalkingTime[(i,j)] 
                            for i,j in cartesian_prod) <= MAX, name='Maxi')
    
    m._cycle = Cycle_x
    m._walk = Walk_y
    m._cbCuts = 0
    #m.setParam("Cuts", C)
    #m.setParam("LiftProjectCuts", L)
    #m.setParam("ZeroHalfCuts", Z)
    #m.setParam('GomoryPasses',G)
    #m.setParam("Presolve", P)
    m.Params.lazyConstraints = 1
    m.optimize(subtourelim)
    m.optimize()
    for v in m.getVars():
        if v.X != 0:
            print('%s %g' % (v.VarName, v.X))
    return m
    

In [15]:
# Callback - use lazy constraints to eliminate sub-tours
# This is an adapted version of the subtour elimination function in the Traveling Salesman Problem example from Gurobi
# https://colab.research.google.com/github/Gurobi/modeling-examples/blob/master/traveling_salesman/tsp_gcl.ipynb


def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of cycle edges selected in the solution
        valsc = model.cbGetSolution(model._cycle)
        
        # make a list of walk edges selected in the solution
        valsm = model.cbGetSolution(model._walk)
        
        valc2 = [(i, j) for i, j in cartesian_prod if valsc[i, j] > 0.5]
        valm2 = [(i, j) for i, j in cartesian_prod if valsm[i, j] > 0.5]
        selected = gp.tuplelist(valc2 + valm2)
        total = len(valc2 + valm2)
        
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        edges = []
        for i in range(len(tour)-1):
            edges.append((tour[i],tour[i+1]))
        edges.append((tour[-1],tour[0]))
        
        edgesc = []
        edgesw = []
        for e in edges:
            if valsc[e] > 0.5:
                edgesc.append(e)
            else:
                edgesw.append(e)
                
        # If the tour length is not the whole length then add a lazy constraint 
        if len(tour) < total:
            model.cbLazy((gp.quicksum(model._cycle[i, j] for (i, j) in edgesc) 
                          + gp.quicksum(model._walk[i, j] for (i, j) in edgesw)) <= len(tour)-1)
            model._cbCuts += 1

# Given a tuplelist of edges, find the shortest subtour
def subtour(edges):
    unvisited = [j for i, j in edges.select('*', '*')]
    cycle = [j for i, j in edges.select('*', '*')] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

In [ ]:
ExpData = pd.DataFrame(columns = ['Runname','Simplex Iterations','RunTime','Number of Lazy Constraints'])

KS = [[0,0,0,1,0],
     [0,0,1,0,0],
     [0,1,0,0,0],
     [0,0,0,0,0],
     [1,1,1,1,0],
     [0,1,1,1,0],
     [0,0,0,1,-1],
     [0,0,1,0,-1],
     [0,1,0,0,-1],
     [0,0,0,0,-1],
     [1,1,1,1,-1],
     [0,1,1,1,-1]]

Names = ['Gomory Only - No Presolve',
        'Zero-Half Only - No Presolve',
        'Lift-Project Only - No Presolve',
        'No Cuts - No Presolve',
         'Every Cut - No Presolve',
         '3 Type Cuts Only - No Presolve',
         'Gomory Only - Presolve',
        'Zero-Half Only - Presolve',
        'Lift-Project Only - Presolve',
        'No Cuts - Presolve',
         'Every Cut - Presolve',
         '3 Type Cuts Only - Presolve',
        ]
     
for i in range(1):
    Name = Names[i]
    K = KS[i]
    z = modeloptimize(K[0],K[1],K[2],K[3],K[4])
    simplex = z.IterCount
    runtime = z.Runtime
    LZ = z._cbCuts
    Name = Names[i]
    ExpData = ExpData.append({'Runname': Name,'Simplex Iterations':simplex,
                              'RunTime':runtime,'Number of Lazy Constraints':LZ},ignore_index=True)

In [23]:
z = modeloptimize(1,1,1,1,-1)
simplex = z.IterCount
runtime = z.Runtime
LZ = z._cbCuts

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 128874 rows, 253472 columns and 1091825 nonzeros
Model fingerprint: 0xc5e627f8
Variable types: 0 continuous, 253472 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Presolve removed 125007 rows and 246688 columns
Presolve time: 0.50s
Presolved: 3867 rows, 6784 columns, 26785 nonzeros
Variable types: 0 continuous, 6784 integer (6784 binary)
Found heuristic solution: objective 3.0000000

Root relaxation: objective 4.266667e+01, 526 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   42.66667    0   30    3.00000   

Best objective 3.300000000000e+01, best bound 3.300000000000e+01, gap 0.0000%

User-callback calls 115527, time in user-callback 117.51 sec
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 128874 rows, 253472 columns and 1091825 nonzeros
Model fingerprint: 0xc5e627f8
Variable types: 0 continuous, 253472 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Presolved: 3867 rows, 6784 columns, 26785 nonzeros

Continuing optimization...


Cutting planes:
  Gomory: 19
  Lift-and-project: 129
  Cover: 39
  MIR: 21
  StrongCG: 1
  Flow cover: 97
  Zero half: 41
  RLT: 22
  Lazy constraints: 57

Explored 47457 nodes (912473 simplex iterations) in 0.26 seconds (0.14 work units)
Thread count was 4 (of 4 available processors)

Solution count 10: 33 32 31 ... 11

Optimal 

Find all the path between the stations in the original data

In [ ]:
alledges = []
for v in z.getVars():
    if v.X > 0.1:
        string = str(v.VarName[10:-1])
        com = (string.find(','))
        start = int(string[0:com])
        end = int(string[com+1:])
        startstation = DataStations[start]
        endstation = DataStations[end]
        alledges.append([start,end])
print(alledges)